In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
train_set=pd.read_csv('./train.csv')
test_set=pd.read_csv('./test.csv')

In [3]:
print(train_set.head())
#print(test_set.head())

  transaction_id     num_var_1  num_var_2  num_var_3     num_var_4  \
0          id_11  2.302632e-08   0.040182        0.0  1.800000e-07   
1          id_33  7.965789e-06   0.157872        0.0  2.105000e-06   
2          id_51  7.828947e-08   0.089140        0.0  3.550000e-07   
3          id_54  7.894737e-08   0.227239        0.0  1.050000e-06   
4          id_62  3.321053e-06   0.160410        0.0  2.105000e-06   

      num_var_5     num_var_6     num_var_7 cat_var_1 cat_var_2  ...    \
0  2.302632e-08  2.368421e-08  1.115205e-08       NaN        ce  ...     
1  2.769737e-07  7.965789e-06  2.433058e-06        da        tn  ...     
2  4.671053e-08  1.052632e-07  4.276014e-07        gf        ce  ...     
3  1.381579e-07  2.190789e-07  1.848054e-08       NaN        ce  ...     
4  2.769737e-07  3.340789e-06  2.152983e-06        da        tn  ...     

  cat_var_34 cat_var_35 cat_var_36 cat_var_37 cat_var_38 cat_var_39  \
0          0          0          0          0          0       

In [4]:
print(train_set.columns)

Index(['transaction_id', 'num_var_1', 'num_var_2', 'num_var_3', 'num_var_4',
       'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_1', 'cat_var_2',
       'cat_var_3', 'cat_var_4', 'cat_var_5', 'cat_var_6', 'cat_var_7',
       'cat_var_8', 'cat_var_9', 'cat_var_10', 'cat_var_11', 'cat_var_12',
       'cat_var_13', 'cat_var_14', 'cat_var_15', 'cat_var_16', 'cat_var_17',
       'cat_var_18', 'cat_var_19', 'cat_var_20', 'cat_var_21', 'cat_var_22',
       'cat_var_23', 'cat_var_24', 'cat_var_25', 'cat_var_26', 'cat_var_27',
       'cat_var_28', 'cat_var_29', 'cat_var_30', 'cat_var_31', 'cat_var_32',
       'cat_var_33', 'cat_var_34', 'cat_var_35', 'cat_var_36', 'cat_var_37',
       'cat_var_38', 'cat_var_39', 'cat_var_40', 'cat_var_41', 'cat_var_42',
       'target'],
      dtype='object')


In [28]:
features=['num_var_1', 'num_var_2', 'num_var_3', 'num_var_4',
       'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_1', 'cat_var_2',
       'cat_var_3', 'cat_var_4', 'cat_var_5', 'cat_var_6', 'cat_var_7',
       'cat_var_8', 'cat_var_9', 'cat_var_10', 'cat_var_11', 'cat_var_12',
       'cat_var_13', 'cat_var_14', 'cat_var_15', 'cat_var_16', 'cat_var_17',
       'cat_var_18', 'cat_var_19', 'cat_var_20']
target=['target']
train=train_set[features]
test=test_set[features]
y=train_set[target]

In [29]:
def handle_strings(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

#df = handle_non_numerical_data(df)
#print(df.head())
train=handle_strings(train)
test=handle_strings(test)

/home/pritam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score

In [31]:
train=preprocessing.scale(train)
test=preprocessing.scale(test)

In [32]:
x_train,x_test,y_train,y_test=train_test_split(train,y,test_size=0.1)

In [33]:
clf=LogisticRegression()

In [34]:
clf.fit(x_train,y_train)

/home/pritam/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
clf.score(x_test,y_test)

0.90965098286434753

In [36]:
predictions=clf.predict(x_test)

In [37]:
print(roc_auc_score(predictions,y_test))

0.903423299768


In [38]:
prediction=clf.predict(test)

In [39]:
ids=test_set['transaction_id']
submission={"target":prediction,"transaction_id":ids}

In [40]:
submission=pd.DataFrame(submission)

In [41]:
x=submission['target']
del submission['target']
submission['target']=x
print(submission)

       transaction_id  target
0                id_1       0
1                id_6       0
2                id_9       0
3               id_14       0
4               id_15       0
5               id_19       0
6               id_20       0
7               id_24       0
8               id_25       0
9               id_28       0
10              id_31       0
11              id_32       0
12              id_37       0
13              id_41       0
14              id_49       0
15              id_50       0
16              id_52       0
17              id_55       0
18              id_56       0
19              id_57       0
20              id_59       0
21              id_60       0
22              id_61       0
23              id_63       0
24              id_70       0
25              id_74       0
26              id_75       0
27              id_77       0
28              id_81       0
29              id_82       0
...               ...     ...
523436      id_872340       0
523437    

In [42]:
submission.to_csv('logistic_reg.csv',index=False)

In [43]:
a=pd.read_csv('./logistic_reg.csv')
print(len(a['target']))
print(a.head())

523466
  transaction_id  target
0           id_1       0
1           id_6       0
2           id_9       0
3          id_14       0
4          id_15       0


In [44]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [45]:
clf=KNeighborsClassifier()

In [46]:
clf.fit(x_train,y_train)

/home/pritam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [47]:
predictions=clf.predict(x_test)

In [48]:
clf.score(x_test,y_test)

0.91443635738437734

In [49]:
predictions=clf.predict(x_test)

In [50]:
holdout_ids = test_set["transaction_id"]
submission_df = {"transaction_id": holdout_ids,
                 "target": prediction}
submission = pd.DataFrame(submission_df)
submission.to_csv("submission1.csv",index=False)
s=pd.read_csv('submission1.csv')
print(s.head())
x=s['target']
del s['target']
print(s.head())
s['target']=x
s.to_csv('submission1.csv',index=False)

   target transaction_id
0       0           id_1
1       0           id_6
2       0           id_9
3       0          id_14
4       0          id_15
  transaction_id
0           id_1
1           id_6
2           id_9
3          id_14
4          id_15


In [ ]:
a=pd.read_csv('submission1.csv')

In [ ]:
print(a.head())